<a href="https://colab.research.google.com/github/hyein-kim-CAU/smart-city/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW1

In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

# Vectorize the sequences
def vectorize_sequences(sequences, dimension=10000):
    results = tf.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results = tf.tensor_scatter_nd_update(results, [[i]], [tf.reduce_sum(tf.one_hot(sequence, depth=dimension), axis=0)])
    return results

x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)

y_train = tf.convert_to_tensor(y_train).numpy().astype("float32")
y_test = tf.convert_to_tensor(y_test).numpy().astype("float32")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:

# Define three models: baseline, L1 regularization, and Dropout
def build_model(model_type='baseline'):
    model = keras.Sequential()
    if model_type == 'l1':
        model.add(layers.Dense(16, activation='relu', input_shape=(10000,),
                               kernel_regularizer=regularizers.l1(0.001)))
    else:
        model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))

    if model_type == 'dropout':
        model.add(layers.Dropout(0.5))

    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:

# Train and record history
histories = {}
for name in ['baseline', 'l1', 'dropout']:
    print(f"Training {name} model...")
    model = build_model(name)
    history = model.fit(x_train, y_train,
                        epochs=20,
                        batch_size=512,
                        validation_split=0.2,
                        verbose=0)
    histories[name] = history


In [ ]:

# Prepare history DataFrame
history_df = pd.DataFrame({
    'epoch': range(1, 21),
    'baseline_val_loss': histories['baseline'].history['val_loss'],
    'l1_val_loss': histories['l1'].history['val_loss'],
    'dropout_val_loss': histories['dropout'].history['val_loss'],
    'baseline_val_acc': histories['baseline'].history['val_accuracy'],
    'l1_val_acc': histories['l1'].history['val_accuracy'],
    'dropout_val_acc': histories['dropout'].history['val_accuracy'],
})
history_df.head()


In [ ]:

# Plotting validation loss and accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_df['epoch'], history_df['baseline_val_loss'], label='Baseline')
plt.plot(history_df['epoch'], history_df['l1_val_loss'], label='L1 Regularization')
plt.plot(history_df['epoch'], history_df['dropout_val_loss'], label='Dropout')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_df['epoch'], history_df['baseline_val_acc'], label='Baseline')
plt.plot(history_df['epoch'], history_df['l1_val_acc'], label='L1 Regularization')
plt.plot(history_df['epoch'], history_df['dropout_val_acc'], label='Dropout')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
